## Fake News Classification Using LSTM

In [7]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

In [2]:
df = pd.read_csv(r"D:\Python DataScience\Fake News Dataset\train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# DROPPING NAN VALUES
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)
y=df['label']

In [6]:
X.shape,y.shape

((18285, 4), (18285,))

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
voc_size=5000 # Declaring a vocabulary size

## One Hot Encoding Representation of the Data

In [13]:
messages=X.copy()
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [15]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [16]:
messages.reset_index(inplace=True)

In [17]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
%time
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Wall time: 0 ns


In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0],onehot_repr[1],onehot_repr[2]

([1655, 3699, 4703, 3730, 2482, 3759, 1205, 4054, 3999, 2469],
 [1749, 3877, 1037, 2283, 1465, 2117, 1411],
 [3536, 2615, 4513, 355])

## Embedding Representation 

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4054 3999 2469]
 [   0    0    0 ... 1465 2117 1411]
 [   0    0    0 ... 2615 4513  355]
 ...
 [   0    0    0 ... 4976 2000 3168]
 [   0    0    0 ...  492  463 4476]
 [   0    0    0 ... 3794  550   84]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1655,
       3699, 4703, 3730, 2482, 3759, 1205, 4054, 3999, 2469])

In [29]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) # one LSTM layer with 100 neurons 
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [34]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 31s 3ms/sample - loss: 0.3371 - accuracy: 0.8397 - val_loss: 0.2061 - val_accuracy: 0.9132
Epoch 2/10
12250/12250 [==============================] - 4s 317us/sample - loss: 0.1410 - accuracy: 0.9457 - val_loss: 0.2040 - val_accuracy: 0.9178
Epoch 3/10
12250/12250 [==============================] - 4s 304us/sample - loss: 0.0993 - accuracy: 0.9638 - val_loss: 0.2339 - val_accuracy: 0.9102
Epoch 4/10
12250/12250 [==============================] - 4s 305us/sample - loss: 0.0712 - accuracy: 0.9760 - val_loss: 0.2713 - val_accuracy: 0.9009
Epoch 5/10
12250/12250 [==============================] - 4s 310us/sample - loss: 0.0469 - accuracy: 0.9848 - val_loss: 0.3038 - val_accuracy: 0.9112
Epoch 6/10
12250/12250 [==============================] - 4s 327us/sample - loss: 0.0368 - accuracy: 0.9874 - val_loss: 0.3856 - val_accuracy: 0.8998
Epoch 7/10
12250/12250 [============================

In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
y_pred=model.predict_classes(X_test)
confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

0.9050538525269263

## Adding a dropout layer 

In [57]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [58]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 7s 574us/sample - loss: 0.3463 - accuracy: 0.8331 - val_loss: 0.1976 - val_accuracy: 0.9157
Epoch 2/10
12250/12250 [==============================] - 3s 247us/sample - loss: 0.1517 - accuracy: 0.9420 - val_loss: 0.1948 - val_accuracy: 0.9186
Epoch 3/10
12250/12250 [==============================] - 3s 241us/sample - loss: 0.1112 - accuracy: 0.9606 - val_loss: 0.2245 - val_accuracy: 0.9114
Epoch 4/10
12250/12250 [==============================] - 3s 252us/sample - loss: 0.0797 - accuracy: 0.9705 - val_loss: 0.2215 - val_accuracy: 0.9180
Epoch 5/10
12250/12250 [==============================] - 3s 244us/sample - loss: 0.0583 - accuracy: 0.9805 - val_loss: 0.3163 - val_accuracy: 0.9107
Epoch 6/10
12250/12250 [==============================] - 3s 243us/sample - loss: 0.0420 - accuracy: 0.9860 - val_loss: 0.2900 - val_accuracy: 0.9085
Epoch 7/10
12250/12250 [===========================

In [60]:
y_pred=model.predict_classes(X_test)
confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

0.9093620546810274